In [ ]:
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split

1. carregamento base

In [ ]:
from google.colab import files

# Isso abre uma janela para você escolher o arquivo no seu computador
uploaded = files.upload()

Saving mimic_iv_ext_onto_sepse_rotulado.parquet to mimic_iv_ext_onto_sepse_rotulado (2).parquet


In [ ]:
df = pd.read_parquet("mimic_iv_ext_onto_sepse_rotulado.parquet")
df.head()

,subject_id,hadm_id,stay_id,charttime,fc,pas,pad,pam,fr,spo,...,leu,pla,ph,uri,hem,ida,pes,alt,sexo,tem_sepse
0,10000032,29079034,39553978,2180-07-23 14:11:00,NaN,84.0,48.0,56.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
1,10000032,29079034,39553978,2180-07-23 14:12:00,91.0,NaN,NaN,NaN,24.0,NaN,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
2,10000032,29079034,39553978,2180-07-23 14:13:00,NaN,NaN,NaN,NaN,NaN,98.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
3,10000032,29079034,39553978,2180-07-23 14:30:00,93.0,95.0,59.0,67.0,21.0,97.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0
4,10000032,29079034,39553978,2180-07-23 15:00:00,94.0,88.0,56.0,64.0,23.0,97.0,...,NaN,NaN,NaN,NaN,NaN,52.0,39.400002,152.0,0,0


- retirar colunas
- fazer janelas de 2 horas pois tera mais pontos no tempo e mais chances de capturar mudancas sutis
- adicionar coluna "janela_index" e "inicio_janela"
- ordenar por paciente, internacao e tempo garante que cada medicao esta na ordem correta dentro de cada internacao
- renan usou min() para definir os inicio_janela, eu optei por floor('2H') pois facilta a analise temporal com janelas uniformes

In [ ]:
# Converte charttime para datetime
df['charttime'] = pd.to_datetime(df['charttime'])

# Ordena os dados por paciente, internação e tempo
df = df.sort_values(['subject_id', 'stay_id', 'charttime'])

# Arredonda charttime para o início da janela de 2 horas
df['inicio_janela'] = df['charttime'].dt.floor('2H')

# Ordena novamente por paciente, internação e início da janela
df = df.sort_values(['subject_id', 'stay_id', 'inicio_janela'])

# Cria o índice da janela por paciente + internação, começando do zero
df['janela_index'] = df.groupby(['subject_id', 'stay_id'])['inicio_janela'].rank(method='dense').astype(int) - 1
# Alternativa:
# df['janela_index'] = df.groupby(['subject_id', 'stay_id'])['inicio_janela'].cumcount()

# Seleciona as colunas finais desejadas
colunas_finais = [
    'subject_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime', 'tem_sepse',
    'fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt'
]

df_final = df[colunas_finais]

# Visualiza as 15 primeiras linhas
df_final.head(15)


/tmp/ipython-input-21-914987155.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df['inicio_janela'] = df['charttime'].dt.floor('2H')


,subject_id,stay_id,janela_index,inicio_janela,charttime,tem_sepse,fc,pas,pad,pam,fr,spo,tem,ida,pes,alt
0,10000032,39553978,0,2180-07-23 14:00:00,2180-07-23 14:11:00,0,NaN,84.0,48.0,56.0,NaN,NaN,NaN,52.0,39.400002,152.0
1,10000032,39553978,0,2180-07-23 14:00:00,2180-07-23 14:12:00,0,91.0,NaN,NaN,NaN,24.0,NaN,NaN,52.0,39.400002,152.0
2,10000032,39553978,0,2180-07-23 14:00:00,2180-07-23 14:13:00,0,NaN,NaN,NaN,NaN,NaN,98.0,NaN,52.0,39.400002,152.0
3,10000032,39553978,0,2180-07-23 14:00:00,2180-07-23 14:30:00,0,93.0,95.0,59.0,67.0,21.0,97.0,NaN,52.0,39.400002,152.0
4,10000032,39553978,0,2180-07-23 14:00:00,2180-07-23 15:00:00,0,94.0,88.0,56.0,64.0,23.0,97.0,NaN,52.0,39.400002,152.0
5,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:00:00,0,105.0,NaN,NaN,NaN,21.0,94.0,NaN,52.0,39.400002,152.0
6,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 16:01:00,0,NaN,91.0,55.0,64.0,NaN,NaN,NaN,52.0,39.400002,152.0
7,10000032,39553978,1,2180-07-23 16:00:00,2180-07-23 17:00:00,0,97.0,95.0,58.0,67.0,20.0,95.0,NaN,52.0,39.400002,152.0
8,10000032,39553978,2,2180-07-23 18:00:00,2180-07-23 18:00:00,0,100.0,86.0,53.0,60.0,21.0,95.0,NaN,52.0,39.400002,152.0
9,10000032,39553978,2,2180-07-23 18:00:00,2180-07-23 19:00:00,0,97.0,93.0,41.0,56.0,16.0,98.0,NaN,52.0,39.400002,152.0


2. serializacao temporal dos dados
- limita a 10 janelas de 2 horas para cada paciente
- pacientes com apenas um janela sao mantidos
- 50920 pacientes com media de 9.8 janelas e desvio padrao baixo
- lstm vai receber sequencias quase uniformes o que melhora a performance

In [ ]:
# Ordena para formar as sequências temporais por paciente
df_serializado_2h = df_final.sort_values(['subject_id', 'stay_id', 'inicio_janela'])

# Filtra os dados para manter apenas as janelas com índice 0 a 9 (ou seja, até 10 janelas por paciente)
df_filtrado_10j = df_serializado_2h.groupby(['subject_id', 'stay_id']).filter(
    lambda x: x['janela_index'].nunique() > 0  # Mantém pacientes com ao menos 1 janela
)
df_filtrado_10j = df_filtrado_10j[df_filtrado_10j['janela_index'] < 10].reset_index(drop=True)

# Estatísticas: quantas janelas por paciente/internação
print(df_filtrado_10j.groupby(['subject_id', 'stay_id'])['janela_index'].nunique().describe())


count    50920.000000
mean         9.835114
std          0.868724
min          1.000000
25%         10.000000
50%         10.000000
75%         10.000000
max         10.000000
Name: janela_index, dtype: float64


In [ ]:
print(df_filtrado_10j.loc[df_filtrado_10j['janela_index'] > 9, 'janela_index'].unique())

[]


- numero de janelas por pacientes

In [ ]:
df_filtrado_10j.groupby(['subject_id', 'stay_id'])['janela_index'].nunique().value_counts()


,count
janela_index,
10,48359
9,688
8,514
7,399
6,273
5,222
4,159
3,154
2,119


3. transformando em formato x,y,mask
- separou a coluna target e as features das de identificacao e tempo


In [ ]:
# ✅ Verificando as colunas disponíveis
print("Colunas disponíveis:")
print(df_filtrado_10j.columns.tolist())

# ❌ Colunas que NÃO são features (identificadores, datas, etc.)
colunas_id = ['subject_id', 'stay_id', 'janela_index', 'charttime', 'inicio_janela']

# 🎯 Coluna target (rótulo de sepse)
target_col = 'tem_sepse'

# ✅ Seleção automática das colunas de features (exames, sinais vitais, etc.)
feature_cols = [col for col in df_filtrado_10j.columns
                if col not in colunas_id + [target_col]]

print(f"\nColunas de features (exames): {len(feature_cols)}")
print(feature_cols)

Colunas disponíveis:
['subject_id', 'stay_id', 'janela_index', 'inicio_janela', 'charttime', 'tem_sepse', 'fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']

Colunas de features (exames): 10
['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']


- X: array de forma (paciente, janela, features)
- y: rotulo 0 ou 1 indicando se tem sepse
- mask: indica quais janelas sao validas (1) ou ausentes (0)
- cria arrays que serao usados como entrada para o lstm

In [ ]:
# Criar lista única de pacientes/internações
print("Criando lista de pacientes únicos...")
pacientes_unicos = df_filtrado_10j.groupby(['subject_id', 'stay_id']).first().reset_index()
print(f"Total de pacientes/internações únicos: {len(pacientes_unicos)}")

# Definir dimensões
n_pacientes = len(pacientes_unicos)
n_janelas = 10  # máximo de janelas
n_features = len(feature_cols)

print(f"Dimensões do array X: ({n_pacientes}, {n_janelas}, {n_features})")

# Inicializar arrays
X = np.zeros((n_pacientes, n_janelas, n_features))
y = np.zeros(n_pacientes)
mask = np.zeros((n_pacientes, n_janelas))

print("Arrays inicializados. Preenchendo dados...")

Criando lista de pacientes únicos...
Total de pacientes/internações únicos: 50920
Dimensões do array X: (50920, 10, 10)
Arrays inicializados. Preenchendo dados...


4. versao sem tratamento
- percorre o array x, y e mask preenchendo os valores com os dados

In [ ]:
# Preencher arrays
for i, (_, paciente_row) in enumerate(pacientes_unicos.iterrows()):
    if i % 5000 == 0:
        print(f"Processando paciente {i}/{n_pacientes}...")

    subject_id = paciente_row['subject_id']
    stay_id = paciente_row['stay_id']

    # Obter dados deste paciente/internação
    dados_paciente = df_filtrado_10j[
        (df_filtrado_10j['subject_id'] == subject_id) &
        (df_filtrado_10j['stay_id'] == stay_id)
    ].sort_values('janela_index')

    # Preencher target y (usar o valor da primeira linha, pois deve ser igual para todas as janelas)
    y[i] = dados_paciente['tem_sepse'].iloc[0]

    # Preencher features X e mask
    for j, (_, janela_row) in enumerate(dados_paciente.iterrows()):
        if j < n_janelas:  # garantir que não excede 10 janelas
            janela_idx = int(janela_row['janela_index'])
            if janela_idx < n_janelas:
                # Preencher features
                X[i, janela_idx, :] = janela_row[feature_cols].values
                # Marcar como válido na mask
                mask[i, janela_idx] = 1

print("✅ Preenchimento concluído!")
print(f"Shape X: {X.shape}")
print(f"Shape y: {y.shape}")
print(f"Shape mask: {mask.shape}")

Processando paciente 0/50920...
Processando paciente 5000/50920...
Processando paciente 10000/50920...
Processando paciente 15000/50920...
Processando paciente 20000/50920...
Processando paciente 25000/50920...
Processando paciente 30000/50920...
Processando paciente 35000/50920...
Processando paciente 40000/50920...
Processando paciente 45000/50920...
Processando paciente 50000/50920...
✅ Preenchimento concluído!
Shape X: (50920, 10, 10)
Shape y: (50920,)
Shape mask: (50920, 10)


5. forward fill version
- recria arrays com tratamento para valores NaN definindo padroes de valores para cada feature
- forward fill: preenche NaN com ultimo valor valido anterior
- backward fill: preenche NaN restantes com o proximo valor valido
- para cada janela preenchida, prrenche o array com os dados corrigidos e se ainda tiver NaN usa o valor padrao

In [ ]:
# 🛠️ CORREÇÃO FINAL: Manter TODAS Features + Forward Fill Inteligente

print("🔧 IMPLEMENTANDO CORREÇÃO MANTENDO TODAS AS FEATURES:")
print("=" * 60)

# MANTER todas as features (incluindo idade se estiver disponível)
print(f"✅ MANTENDO TODAS AS {len(feature_cols)} FEATURES:")
print(f"   {feature_cols}")

# Recriar arrays mantendo todas as features
print(f"\n🔄 RECRIANDO ARRAYS COM TRATAMENTO INTELIGENTE DE NaN:")
X_novo = np.zeros((n_pacientes, n_janelas, n_features))
y_novo = np.zeros(n_pacientes)
mask_novo = np.zeros((n_pacientes, n_janelas))

print(f"   Dimensões mantidas: ({n_pacientes}, {n_janelas}, {n_features})")

# Valores normais para cada feature (para casos extremos de NaN)
valores_normais = {
    'fc': 70,      # Frequência cardíaca normal (bpm)
    'pas': 120,    # Pressão sistólica normal (mmHg)
    'pad': 80,     # Pressão diastólica normal (mmHg)
    'pam': 93,     # Pressão arterial média normal (mmHg)
    'fr': 16,      # Frequência respiratória normal (rpm)
    'spo': 98,     # Saturação de oxigênio normal (%)
    'tem': 36.5,   # Temperatura normal (°C)
    'pes': 70,     # Peso médio (kg)
    'ida': 65      # Idade mediana (anos)
}

print(f"📋 Valores normais definidos para backup: {valores_normais}")

# Loop corrigido com forward fill por paciente
print(f"\n🔄 Processando pacientes com forward fill...")
for i, (_, paciente_row) in enumerate(pacientes_unicos.iterrows()):
    if i % 10000 == 0:
        print(f"   Processando paciente {i}/{n_pacientes}...")

    subject_id = paciente_row['subject_id']
    stay_id = paciente_row['stay_id']

    # Obter dados deste paciente/internação
    dados_paciente = df_filtrado_10j[
        (df_filtrado_10j['subject_id'] == subject_id) &
        (df_filtrado_10j['stay_id'] == stay_id)
    ].sort_values('janela_index').copy()

    # Preencher target y
    y_novo[i] = dados_paciente['tem_sepse'].iloc[0]

    # Forward fill POR PACIENTE para cada feature
    for feature in feature_cols:
        dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
        dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')

    # Processar por janela
    for _, janela_row in dados_paciente.iterrows():
        janela_idx = int(janela_row['janela_index'])

        if janela_idx < n_janelas:
            # Preencher features
            for j, feature in enumerate(feature_cols):
                valor = janela_row[feature]

                # Se ainda é NaN após forward/backward fill, usar valor normal
                if pd.isna(valor):
                    valor = valores_normais.get(feature, 0)

                X_novo[i, janela_idx, j] = valor

            # Marcar janela como válida
            mask_novo[i, janela_idx] = 1

print(f"\n✅ PROCESSAMENTO CONCLUÍDO!")

# Verificar resultados
janelas_por_paciente_novo = np.sum(mask_novo, axis=1)
print(f"\n📊 ESTATÍSTICAS FINAIS:")
print(f"   Média de janelas por paciente: {np.mean(janelas_por_paciente_novo):.2f}")
print(f"   Min janelas: {np.min(janelas_por_paciente_novo)}")
print(f"   Max janelas: {np.max(janelas_por_paciente_novo)}")
print(f"   Pacientes com 0 janelas: {np.sum(janelas_por_paciente_novo == 0)}")
print(f"   Valores NaN em X_novo: {np.sum(np.isnan(X_novo))}")
print(f"   X_novo - Min: {np.min(X_novo):.2f}, Max: {np.max(X_novo):.2f}")

# Verificar distribuição de classes
print(f"\n🎯 DISTRIBUIÇÃO DE CLASSES:")
print(f"   Com sepse: {np.sum(y_novo == 1)} ({np.mean(y_novo)*100:.1f}%)")
print(f"   Sem sepse: {np.sum(y_novo == 0)} ({(1-np.mean(y_novo))*100:.1f}%)")

# Atualizar variáveis globais
X = X_novo
y = y_novo
mask = mask_novo

print(f"\n🎉 SUCESSO!")
print(f"   ✅ Todas as {n_features} features mantidas")
print(f"   ✅ Forward fill aplicado por paciente")
print(f"   ✅ Valores normais para casos extremos")
print(f"   ✅ Sem remoção de features importantes")
print(f"   📈 Média de janelas: {np.mean(janelas_por_paciente_novo):.2f}")

if np.mean(janelas_por_paciente_novo) > 6:
    print(f"   🎯 Média próxima do esperado (~7.9)!")
else:
    print(f"   ⚠️  Média ainda baixa - pode haver problema estrutural nos dados")

🔧 IMPLEMENTANDO CORREÇÃO MANTENDO TODAS AS FEATURES:
✅ MANTENDO TODAS AS 10 FEATURES:
   ['fc', 'pas', 'pad', 'pam', 'fr', 'spo', 'tem', 'ida', 'pes', 'alt']

🔄 RECRIANDO ARRAYS COM TRATAMENTO INTELIGENTE DE NaN:
   Dimensões mantidas: (50920, 10, 10)
📋 Valores normais definidos para backup: {'fc': 70, 'pas': 120, 'pad': 80, 'pam': 93, 'fr': 16, 'spo': 98, 'tem': 36.5, 'pes': 70, 'ida': 65}

🔄 Processando pacientes com forward fill...
   Processando paciente 0/50920...


Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dado

   Processando paciente 10000/50920...


Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dado

   Processando paciente 20000/50920...


...

Streaming output truncated to the last 5000 lines.
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dado

   Processando paciente 50000/50920...


/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipy


✅ PROCESSAMENTO CONCLUÍDO!

📊 ESTATÍSTICAS FINAIS:
   Média de janelas por paciente: 9.84
   Min janelas: 1.0
   Max janelas: 10.0
   Pacientes com 0 janelas: 0
   Valores NaN em X_novo: 0
   X_novo - Min: 0.00, Max: 250.00

🎯 DISTRIBUIÇÃO DE CLASSES:
   Com sepse: 7062 (13.9%)
   Sem sepse: 43858 (86.1%)

🎉 SUCESSO!
   ✅ Todas as 10 features mantidas
   ✅ Forward fill aplicado por paciente
   ✅ Valores normais para casos extremos
   ✅ Sem remoção de features importantes
   📈 Média de janelas: 9.84
   🎯 Média próxima do esperado (~7.9)!


/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipython-input-28-1955790960.py:53: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='ffill')
/tmp/ipython-input-28-1955790960.py:54: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dados_paciente[feature] = dados_paciente[feature].fillna(method='bfill')
/tmp/ipy

In [ ]:
# Verificar os dados criados
print("📊 VERIFICAÇÃO DOS DADOS CRIADOS:")
print(f"Total de pacientes: {len(y)}")
print(f"Com sepse: {np.sum(y == 1)}")
print(f"Sem sepse: {np.sum(y == 0)}")
print(f"Proporção com sepse: {np.mean(y):.3f}")

print(f"\n📈 ESTATÍSTICAS DAS SEQUÊNCIAS:")
janelas_por_paciente = np.sum(mask, axis=1)
print(f"Média de janelas por paciente: {np.mean(janelas_por_paciente):.2f}")
print(f"Min janelas: {np.min(janelas_por_paciente)}")
print(f"Max janelas: {np.max(janelas_por_paciente)}")

print(f"\n🔍 VERIFICAÇÃO DE VALORES:")
print(f"X - Min: {np.min(X):.2f}, Max: {np.max(X):.2f}")
print(f"Valores NaN em X: {np.sum(np.isnan(X))}")
print(f"Valores NaN em y: {np.sum(np.isnan(y))}")

# Salvar no formato .npz (igual ao notebook FASE 5)
caminho_arquivo = r"C:\Users\beatr\Documents\mimic_X_y_mask_10j_2h.npz"
np.savez_compressed(
    caminho_arquivo,
    X=X,
    y=y,
    mask=mask
)

print(f"\n💾 Arquivo salvo: {caminho_arquivo}")

📊 VERIFICAÇÃO DOS DADOS CRIADOS:
Total de pacientes: 50920
Com sepse: 7062
Sem sepse: 43858
Proporção com sepse: 0.139

📈 ESTATÍSTICAS DAS SEQUÊNCIAS:
Média de janelas por paciente: 9.84
Min janelas: 1.0
Max janelas: 10.0

🔍 VERIFICAÇÃO DE VALORES:
X - Min: 0.00, Max: 250.00
Valores NaN em X: 0
Valores NaN em y: 0

💾 Arquivo salvo: C:\Users\beatr\Documents\mimic_X_y_mask_10j_2h.npz


In [ ]:
# Tratar valores NaN (substituir por 0 ou média)
print("🔧 TRATANDO VALORES NaN...")
print(f"NaN antes: {np.sum(np.isnan(X))}")

# Substituir NaN por 0 (ou você pode usar a média)
X_clean = np.nan_to_num(X, nan=0.0)

print(f"NaN depois: {np.sum(np.isnan(X_clean))}")
print(f"X_clean - Min: {np.min(X_clean):.2f}, Max: {np.max(X_clean):.2f}")

# Salvar versão limpa
caminho_arquivo_clean = "C:\\Users\\beatr\\Documents\\mimic_X_y_mask_10j_2h_clean.npz"
np.savez_compressed(
    caminho_arquivo_clean,
    X=X_clean,
    y=y,
    mask=mask
)

print(f"\n💾 Arquivo limpo salvo: {caminho_arquivo_clean}")

🔧 TRATANDO VALORES NaN...
NaN antes: 0
NaN depois: 0
X_clean - Min: 0.00, Max: 250.00

💾 Arquivo limpo salvo: C:\Users\beatr\Documents\mimic_X_y_mask_10j_2h_clean.npz


Normalizacao:
- normaliza as features para o intervalo [0,1]
- essa normalizacao esta estranha pois deu min: -2.439 que indica que algum valor fora do intervalo foi transformado
- proximos passos: esperar pipeline do tiago com normalizacao corrigida para aplicar lstm com janelas de diferentes horas

In [ ]:
# 📊 NORMALIZAÇÃO DOS DADOS (MinMaxScaler)
from sklearn.preprocessing import MinMaxScaler
import joblib

print("🔧 INICIANDO NORMALIZAÇÃO...")

# Carregar dados limpos para normalizar
data_clean = np.load("C:\\Users\\beatr\\Documents\\mimic_X_y_mask_10j_2h_clean.npz")
X = data_clean['X']
y = data_clean['y']
mask = data_clean['mask']

# Criar cópia para normalização
X_norm = np.copy(X)
n_pacientes, n_janelas, n_features = X.shape
print(f"Dados originais: {X.shape} - {n_pacientes} pacientes, {n_janelas} janelas, {n_features} features")

# Lista para armazenar os scalers de cada feature
scalers = []

# Normalizar cada feature separadamente
for i in range(n_features):
    scaler = MinMaxScaler()

    # Reformatar para (n_samples, 1) → necessário para o fit
    feature_data = X[:, :, i].reshape(-1, 1)  # shape (n_pacientes*10, 1)

    # Fazer fit apenas nos dados válidos (não-zero da mask)
    mask_flat = mask.reshape(-1)
    feature_data_valid = feature_data[mask_flat == 1]

    if len(feature_data_valid) > 0:
        scaler.fit(feature_data_valid.reshape(-1, 1))

        # Aplicar transformação em todos os dados
        X_norm[:, :, i] = scaler.transform(feature_data).reshape(n_pacientes, n_janelas)

    scalers.append(scaler)

print(f"✅ Normalização concluída!")
print(f"X_norm shape: {X_norm.shape}")
print(f"X_norm - Min: {np.min(X_norm):.3f}, Max: {np.max(X_norm):.3f}")

# Salvar dados normalizados
caminho_norm = "C:\\Users\\beatr\\Documents\\mimic_X_y_mask_10j_2h_clean_norm.npz"
np.savez_compressed(caminho_norm, X=X_norm, y=y, mask=mask)

print(f"💾 Dados normalizados salvos: {caminho_norm}")
print("🎯 Pronto para balanceamento!")

🔧 INICIANDO NORMALIZAÇÃO...
Dados originais: (50920, 10, 10) - 50920 pacientes, 10 janelas, 10 features
✅ Normalização concluída!
X_norm shape: (50920, 10, 10)
X_norm - Min: -2.439, Max: 1.000
💾 Dados normalizados salvos: C:\Users\beatr\Documents\mimic_X_y_mask_10j_2h_clean_norm.npz
🎯 Pronto para balanceamento!


balanceamento:
- cria subconjunto balanceado com o mesmo numero de pacientes com e sem sepse para evitar que o modelo aprenda apenas a prever a maioria (sem sepse)

In [ ]:
# 📥 Carregar dados normalizados
data = np.load("C:\\Users\\beatr\\Documents\\mimic_X_y_mask_10j_2h_clean_norm.npz")
X = data['X']
y = data['y']
mask = data['mask']

# 🎯 Encontrar índices das classes
idx_sepse = np.where(y == 1)[0]
idx_nsepse = np.where(y == 0)[0]

# ⚖️ Balancear: mesmo número das duas classes
n = min(len(idx_sepse), len(idx_nsepse))
idx_balanced = np.concatenate([idx_sepse[:n], idx_nsepse[:n]])
np.random.shuffle(idx_balanced)

# 📦 Selecionar subconjunto balanceado
X_bal = X[idx_balanced]
y_bal = y[idx_balanced]
mask_bal = mask[idx_balanced]

# 💾 Salvar arquivo balanceado
np.savez_compressed(
    "C:\\Users\\beatr\\Documents\\mimic_X_y_mask_10j_2h_bal.npz",
    X=X_bal,
    y=y_bal,
    mask=mask_bal
)

print("✅ Arquivo balanceado salvo com sucesso.")

✅ Arquivo balanceado salvo com sucesso.
